In [ ]:
# Import Dependencies

from typing import TypedDict
from langgraph.graph import StateGraph, END

In [3]:
class AgentState(TypedDict): # ---> our Schema
    number1:int
    operation:str
    number2:int
    result:float

In [ ]:
# Create Nodes

def adder(state:AgentState) -> AgentState:
    """This node add sum two numbers"""

    state["result"] = state["number1"] + state["number2"]
    return state

def subtractor(state:AgentState) -> AgentState:
    """This node subtracts two numbers"""

    state["result"] = state["number1"] - state["number2"]
    return state

def multiplier(state:AgentState) -> AgentState:
    """This node multiplies two numbers"""

    state["result"] = state["number1"] * state["number2"]
    return state

def divider(state:AgentState) -> AgentState:
    """This node divides two numbers"""

    if state["number2"] == 0:
        raise ValueError("Cannot divide by zero")
    state["result"] = state["number1"] / state["number2"]
    return state


def decide_next_node(state:AgentState) -> AgentState:
    """This node will select the next node of the graph"""

    if state["operation"] == "+":
        return "addition_operation"
    
    elif state["operation"] == "-":
        return "subtraction_operation"
    elif state["operation"] == "*":
        return "multiplication_operation"
    elif state["operation"] == "/":
        return "division_operation"

In [ ]:
graph = StateGraph(AgentState)

# Add Nodes to Graph

graph.add_node("adder", adder)
graph.add_node("subtractor", subtractor)
graph.add_node("multiplier", multiplier)
graph.add_node("divider", divider)
graph.add_node("router", lambda state: state)  # Router node

graph.add_edge(
    "router",
    decide_next_node,
    {   # Edge:Node                   
        "addition_operation": "adder",  
        "subtraction_operation": "subtractor",
        "multiplication_operation": "multiplier",
        "division_operation": "divider",
    }
)

graph.add_edge("adder", END)
graph.add_edge("subtractor", END)
graph.add_edge("multiplier", END)
graph.add_edge("divider", END)


app=graph.compile()



In [ ]:


from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))



In [ ]:


initial_state_1 = AgentState(number1 = 10, operation="-", number2 = 5)
print(app.invoke(initial_state_1))



In [ ]:


# This way still works!

result = app.invoke({"number1": 10, "operation": "-", "number2": 5})
print(result)

